In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re 
from urllib.request import urlopen
import pandas as pd
import numpy as np

In [13]:
# InstagramCrawling1에서 저장된 카페별 csv 파일 로드
df = pd.read_csv('./csv_file/김메주와고양이들.csv', index_col=[0])
df

,name,content,date,like,place,tags,link
0,김메주와고양이들,메주가 디자인 한 야옹컵과 고양이같은하ᄅ...,2020-11-27,like,NaN,[],https://www.instagram.com/here.mejoocats/
1,김메주와고양이들,자이언트 휴지도 산타모자 썼어요🎄🎅 같이 사진 찍고 가는 거 잊지 마세용!,2020-11-21,0 like,NaN,[],https://www.instagram.com/here.mejoocats/
2,김메주와고양이들,마감 때만 오던 오들이가 이젠 손님들 아...,2020-09-27,9 like,Cafe 김메주와 고양이들,[],https://www.instagram.com/here.mejoocats/
3,김메주와고양이들,가을이 끝나가요..🍂 가을 시즌 메뉴인 먼지쑥라떼도 곧 떠난다는 말.... 먼지쑥라...,2020-11-17,2 like,NaN,[],https://www.instagram.com/here.mejoocats/
4,김메주와고양이들,A4 한장이면 고양이 하나 거뜬히 잡져,2020-09-15,8 like,NaN,[],https://www.instagram.com/mejooandcats/
...,...,...,...,...,...,...,...
995,김메주와고양이들,_나한텐 그냥 언닌데 ㅋㅋㅋㅋㅋ무슨 연예인이 따로 없구만_드디어 와본 언니카페손님이...,2018-11-10,ike,NaN,"['#김메주와고양이들..그릇닦으심ㅋㅋㅋㅋㅋㅋ', '#설겆이칼크이해라', '#밥먹으러...",https://www.instagram.com/__.j__j/
996,김메주와고양이들,18.11.10여기가 핫한 곳이라면서요? ㅎㅎ#김메주와고양이들,2018-11-10,ike,NaN,['#김메주와고양이들'],https://www.instagram.com/leeg_rootun07/
997,김메주와고양이들,메주님 카페 처음 방문했습니다.인테리어 좋고 무엇보다 연유라떼 너무 맛있습니다. 그...,2018-11-10,ike,NaN,"['#김메주와고양이들', '#김메주', '#김메주와고양이들카페', '#먼지', '#...",https://www.instagram.com/ironfist_hj_theanima...
998,김메주와고양이들,먼지ㅠㅠ #김메주와고양이들 #먼봉휴요 #먼지 #고양이 #김메주와고양이들굿즈 #🐈,2018-11-10,ike,NaN,"['#김메주와고양이들', '#먼봉휴요', '#먼지', '#고양이', '#김메주와고양...",https://www.instagram.com/soonzzzzi/


In [ ]:
# link 컬럼에서 중복제외한 링크 df 만들기
link_df = df['link'].value_counts().reset_index(name='count')
link_df.rename(columns={'index':'link'}, inplace=True)
link_df

In [4]:
# 함수를 사용하면 팔로워수와 팔로우 수를 추가해줍니다.

def get_follower(df, ID, pw):
    df['follower'] = ''
    df['follow'] = ''
    driver = webdriver.Chrome('./chromedriver')
    url_log = 'https://www.instagram.com/accounts/login/'
    #2. 로그인 하기
    driver.get(url_log)
    time.sleep(2)
    driver.find_element_by_name("username").send_keys(ID)
    driver.find_element_by_name("password").send_keys(pw)
    driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()
    time.sleep(3)
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    for i ,j in enumerate(df['link']):
        driver.get(j)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html)
        # 팔로워 , 팔로우 숫자 입력
        follower = soup.select('span.g47SY ')[1].text.replace(',',"").replace('천','000').replace('백만', '000000000').replace('.',"")
        follow = soup.select('span.g47SY ')[2].text.replace(',',"").replace('천','000').replace('백만', '000000000').replace('.',"")
        df['follower'][i] = follower
        df['follow'][i] = follow
        print(i,'/',len(df))
        
    driver.quit()
    return df

In [9]:
# 팔로워/팔로우수 크롤링은 한번에 성공하는 경우가 드물다.
# 에러 발생시마다 df를 개별적으로 변수 선언후, 다시 크롤링.
# 1차 크롤링
get_follower(link_df, 'input_ID', 'input_PW')

In [ ]:
# 마지막 에러 화면 확인 먼저하여서, 해당 link가 아예 없는 페이지는 아닌지 먼저 확인
# 없어진 페이지라면 해당 링크는 다음 크롤링을 할때 제외시켜야함.
link_df_2 = link_df_2[1:]
link_df_2.reset_index(drop=True, inplace=True)
link_df_2

In [17]:
# 크롤링 완료된 df 별도 저장
link_df = link_df[:228]
link_df.to_csv('./follow_follower_data/김메주_link_df.csv')
link_df 

In [41]:
# 2차 크롤링
get_follower(link_df_2,'input_ID', 'input_PW')

In [ ]:
# 마지막 에러 화면 확인 먼저하여서, 해당 link가 아예 없는 페이지는 아닌지 먼저 확인
# 없어진 페이지라면 해당 링크는 다음 크롤링을 할때 제외시켜야함.
link_df_3 = link_df_2[192:]
link_df_3.reset_index(drop=True, inplace=True)
link_df_3

In [40]:
# 크롤링 완료된 df 별도 저장
link_df_2 = link_df_2[:192]
link_df_2.to_csv('./follow_follower_data/김메주_link_df_2.csv')
link_df_2 

In [51]:
# 3차 크롤링
get_follower(link_df_3, 'input_ID', 'input_PW')

In [71]:
# 마지막 에러 화면 확인 먼저하여서, 해당 link가 아예 없는 페이지는 아닌지 먼저 확인
# 없어진 페이지라면 해당 링크는 다음 크롤링을 할때 제외시켜야함.
link_df_4 = link_temp[484:]
link_df_4.reset_index(drop=True, inplace=True)
link_df_4

In [69]:
# 크롤링 완료된 df 별도 저장
link_df_3 = link_df_3[:61]
link_df_3.to_csv('./follow_follower_data/김메주_link_df_3.csv')
link_df_3

In [70]:
# 4차 크롤링
get_follower(link_df_4, 'input_ID', 'input_PW')

0 / 170
1 / 170
2 / 170
3 / 170
4 / 170
5 / 170
6 / 170
7 / 170
8 / 170
9 / 170
10 / 170
11 / 170
12 / 170
13 / 170
14 / 170
15 / 170
16 / 170
17 / 170
18 / 170
19 / 170
20 / 170
21 / 170
22 / 170
23 / 170
24 / 170
25 / 170
26 / 170
27 / 170
28 / 170
29 / 170
30 / 170
31 / 170
32 / 170
33 / 170
34 / 170
35 / 170
36 / 170
37 / 170
38 / 170
39 / 170
40 / 170
41 / 170
42 / 170
43 / 170
44 / 170
45 / 170
46 / 170
47 / 170
48 / 170
49 / 170
50 / 170
51 / 170
52 / 170
53 / 170
54 / 170
55 / 170
56 / 170
57 / 170
58 / 170
59 / 170
60 / 170
61 / 170
62 / 170
63 / 170
64 / 170
65 / 170
66 / 170
67 / 170
68 / 170
69 / 170
70 / 170
71 / 170
72 / 170
73 / 170
74 / 170
75 / 170
76 / 170
77 / 170
78 / 170
79 / 170
80 / 170
81 / 170
82 / 170
83 / 170
84 / 170
85 / 170
86 / 170
87 / 170
88 / 170
89 / 170
90 / 170
91 / 170
92 / 170
93 / 170
94 / 170
95 / 170
96 / 170
97 / 170
98 / 170
99 / 170
100 / 170
101 / 170
102 / 170
103 / 170
104 / 170
105 / 170
106 / 170
107 / 170
108 / 170
109 / 170
110 / 170


,link,count,follower,follow
0,https://www.instagram.com/yun.lael_junghwa_sk/,1,4552,2720
1,https://www.instagram.com/jh.p_popo/,1,10,41
2,https://www.instagram.com/nori__pine/,1,225,278
3,https://www.instagram.com/sonar_lee2/,1,256,286
4,https://www.instagram.com/ji_hiyam_love/,1,65,84
...,...,...,...,...
165,https://www.instagram.com/kkwang_8o8/,1,152,213
166,https://www.instagram.com/everyday_joyful/,1,6,103
167,https://www.instagram.com/eun_ji__10/,1,77,55
168,https://www.instagram.com/o.o_butter/,1,37,331


In [73]:
# 마지막 크롤링 완료 df 저장
link_df_4.to_csv('./follow_follower_data/김메주_link_df_4.csv')

In [6]:
# 모든 df 합치기
frames = [link_df, link_df_1, link_df_2, link_df_3, link_df_4]
result = pd.concat(frames)

In [9]:
# 합친 df 저장
result.reset_index(drop=True, inplace=True)
result.to_csv('./follow_follower_data/김메주_result.csv')
result

,link,count,follower,follow
0,https://www.instagram.com/cap.tur3_/,83,625,3.0
1,https://www.instagram.com/here.mejoocats/,25,108k,2.0
2,https://www.instagram.com/rukajee_pom/,19,231,231.0
3,https://www.instagram.com/baikyoon2/,17,136,149.0
4,https://www.instagram.com/kim_lovo/,15,895,362.0
...,...,...,...,...
647,https://www.instagram.com/kkwang_8o8/,1,152,213.0
648,https://www.instagram.com/everyday_joyful/,1,6,103.0
649,https://www.instagram.com/eun_ji__10/,1,77,55.0
650,https://www.instagram.com/o.o_butter/,1,37,331.0


In [20]:
# InstagramCrawling1에서 저장된 카페별 csv 파일에 merge.
new = pd.merge(df, result, on="link", how='left')
new.to_csv('./follow_follower_data/김메주_final.csv')